In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load quarterly data from FI_T5(Merge Query).xlsx
quarterly_data = pd.read_excel('D:/MG/！CUHKSZ/~！大二 下/FIN3080/Assignments/A2/A2_FI_T5(Merge Query).xlsx', 
                              dtype={'Stkcd': str})

# Filter for consolidated statements (Typrep == 'A')
quarterly_data = quarterly_data[quarterly_data['Typrep'] == 'A']

# Select relevant columns and rename Net Assets per Share
quarterly_data = quarterly_data[['Stkcd', 'Accper', 'F091001A']]
quarterly_data.rename(columns={'F091001A': 'BVPS'}, inplace=True)

# Convert Accper to datetime
quarterly_data['Accper'] = pd.to_datetime(quarterly_data['Accper'])

KeyError: 'Typrep'

In [ ]:
# Load monthly data from TRD_Mnth.xlsx
monthly_data = pd.read_excel('D:/MG/！CUHKSZ/~！大二 下/FIN3080/Assignments/A2/A2_TRD_Mnth.xlsx', 
                             dtype={'Stkcd': str})

# Select relevant columns
monthly_data = monthly_data[['Stkcd', 'Trdmnt', 'Mclsprc']]

# Convert Trdmnt to datetime and set to the last day of the month
monthly_data['Trdmnt'] = pd.to_datetime(monthly_data['Trdmnt'], format='%Y-%m') + pd.offsets.MonthEnd(0)

# Filter data for January 2010 to December 2024
monthly_data = monthly_data[(monthly_data['Trdmnt'] >= '2010-01-31') & 
                            (monthly_data['Trdmnt'] <= '2024-12-31')]

In [ ]:
# Define a function to merge monthly and quarterly data for each stock
def merge_asof_group(month_group, quarter_group):
    month_group = month_group.sort_values('Trdmnt')
    quarter_group = quarter_group.sort_values('Accper')
    merged = pd.merge_asof(month_group, quarter_group, 
                          left_on='Trdmnt', right_on='Accper', 
                          direction='backward')
    return merged

# Perform the merge for each stock
merged_list = []
for stkcd, month_group in monthly_data.groupby('Stkcd'):
    quarter_group = quarterly_data[quarterly_data['Stkcd'] == stkcd]
    if not quarter_group.empty:
        merged_group = merge_asof_group(month_group, quarter_group)
        merged_list.append(merged_group)

# Concatenate all merged groups into a single DataFrame
merged_df = pd.concat(merged_list)

In [ ]:
# Calculate P/B ratio as Monthly Closing Price / Net Assets per Share
merged_df['PB_ratio'] = merged_df['Mclsprc'] / merged_df['BVPS']

# Drop rows where PB_ratio is NaN (e.g., due to missing BVPS)
merged_df = merged_df.dropna(subset=['PB_ratio'])